# 7. TopNEXt (Intensity Non-overlap)

This notebook shows how to run Intensity Non-overlap, a controller from the TopNEXt framework built on top of ViMMS.
Other controllers under TopNEXt can be run in nearly the same way.

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../..')

In [4]:
from pathlib import Path
import os

In [5]:
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import IntensityNonOverlapController
from vimms.Environment import Environment
from vimms.Common import POSITIVE, load_obj, set_log_level_warning, set_log_level_debug
from vimms.Box import BoxGrid
from vimms.Roi import RoiBuilderParams
from vimms.BoxManager import BoxSplitter, BoxManager

### Load the data

Load pre-processed chemical data. Refer to [this notebook](https://github.com/glasgowcompbio/vimms/blob/master/demo/01.%20Data/02.%20Extracting%20Chemicals%20from%20an%20mzML%20file.ipynb) to see how to generate this.

In [6]:
data_dir = os.path.abspath(os.path.join(os.getcwd(),'..','..','tests','fixtures'))
dataset = load_obj(os.path.join(data_dir, 'beer_compounds.p'))

### Run Intensity Non-overlap Controller

In [7]:
rt_range = [(0, 1440)]
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]

In [8]:
out_dir = os.path.join(os.getcwd(), 'results')

In [9]:
isolation_width = 1  # the isolation window in Dalton around a selected precursor ion
N = 10
rt_tol = 15
mz_tol = 10
min_ms1_intensity = 5000
min_roi_intensity = 0
min_roi_length = 1
ionisation_mode = POSITIVE
rt_box_size, mz_box_size = 1, 0.3

In [10]:
# set the log level to WARNING so we don't see too many messages when environment is running
set_log_level_warning()

1

Data structures that persists across injections

In [11]:
bg = BoxGrid(rt_range[0][0], rt_range[0][1], rt_box_size, 0, 3000, mz_box_size)
grid = BoxManager(
    box_geometry=bg,
    box_splitter=BoxSplitter(split=True)
)

Simulate a repeated injection of the same beer sample three times. Each time the controller is used to acquire data that it hasn't acquired efore. Simulated results are saved to the following .mzML file and can be viewed in tools like ToppView or using other mzML file viewers.

In [12]:
for i in range(3): # repeat simulated injection three times
    
    print(i)
    
    # create a mass spec object
    mass_spec = IndependentMassSpectrometer(POSITIVE, dataset)

    # create the Intensity Non-overlap controller
    roi_params = RoiBuilderParams(min_roi_length=min_roi_length, min_roi_intensity=min_roi_intensity)
    controller = IntensityNonOverlapController(ionisation_mode, isolation_width, N, mz_tol,
                                               rt_tol, min_ms1_intensity, roi_params, grid)

    # create an environment to run both the mass spec and controller
    out_file = 'intensity_nonoverlap_controller_%d.mzML' % i
    env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True, 
                      out_dir=out_dir, out_file=out_file)

    # run the simulation
    env.run()

0


  0%|          | 0/1440 [00:00<?, ?it/s]

1


  0%|          | 0/1440 [00:00<?, ?it/s]

2


  0%|          | 0/1440 [00:00<?, ?it/s]